In [1]:
!which jupyter
import pandas as pd
import numpy as np
import datetime
import spacy
import en_core_web_sm
#from spacy import displacy

/home/ubuntu/thesis_env2/bin/jupyter


In [3]:
df = pd.read_csv('./data/covid19_articles_20201231.csv')

# Date to datetime
df['date'] = pd.to_datetime(df['date'])

# Drop duplicates
num_articles_pre_drop = len(df)
df = df.drop_duplicates(subset='content')
print('no. articles dropped: ', num_articles_pre_drop - len(df))
print('no. of articles: ', len(df))

# Drop articles where text longer than 1,000,000 characters
df = df[df['content'].apply(lambda x: len(x) <= 1000000)].copy()
print('no. of articles after dropping long articles: ', len(df))

print(df.info())

no. articles dropped:  127
no. of articles:  368920
no. of articles after dropping long articles:  368918
<class 'pandas.core.frame.DataFrame'>
Int64Index: 368918 entries, 0 to 369046
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   author      181789 non-null  object        
 1   date        368918 non-null  datetime64[ns]
 2   domain      368918 non-null  object        
 3   title       368833 non-null  object        
 4   url         368918 non-null  object        
 5   content     368918 non-null  object        
 6   topic_area  368918 non-null  object        
dtypes: datetime64[ns](1), object(6)
memory usage: 22.5+ MB
None


In [4]:
df.index

Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            369037, 369038, 369039, 369040, 369041, 369042, 369043, 369044,
            369045, 369046],
           dtype='int64', length=368918)

In [5]:
# Get index numbers of articles to label
np.random.seed(777)
article_indices = np.random.choice(df.index, size=1000, replace=False)
article_indices

array([272801, 200452, 360615, 348240, 150912,   5919,  30166, 136359,
       208183,  23000, 330542,  58596, 286411, 261343, 113511,  42006,
       102479, 241927, 341037, 291682, 212144, 302846, 110625, 201788,
       170802, 159684, 333166, 220772, 237146, 129492, 313847, 156620,
        36890, 340034, 163144, 360941,  55550, 347657,  47979, 182837,
       134691, 160045, 286915, 278396, 233365, 286431,  83397, 166869,
       246232, 308226,  84557, 203388, 170380, 213355, 158274,  76833,
       156025, 264723, 117743,  94017,  47533,  19442, 356015, 167369,
       157731, 221842, 126826, 194484, 140473, 190886, 286245, 165741,
       243097, 225028, 116166, 202311,  50746, 323505,  24801, 306356,
       278877, 232675,  35740,  62652, 148690, 266123, 204738, 115954,
        23313, 190061, 141887, 187155, 171153, 169507, 354893, 160755,
       187739,  89199, 326662, 129575,  41302, 264705, 229588, 249778,
       245717, 174209, 152067, 205808,   7392, 190235, 107862, 276782,
      

In [6]:
# Load file with labels
file_latest = 'df_lab_latest.csv'
df_lab = pd.read_csv('./data/' + file_latest, index_col=0)
print(sum(df_lab['org_names_listed'].isna()))

0


In [7]:
df_lab

,author,date,domain,title,url,content,topic_area,org_names_listed,org_names_listed_uo,listed_count,listed_uo_count,listing_country,listing_country_count
272801,NaN,2020-08-15,finance.yahoo,Julen Lopetegui: We must be at our best to bea...,https://finance.yahoo.com/news/julen-lopetegui...,Head coach Julen Lopetegui knows Sevilla will ...,business,[],[],0,0,[],0
200452,NaN,2020-07-11,finance.yahoo,Gallagher Premiership teams will not be punish...,https://uk.finance.yahoo.com/news/gallagher-pr...,Gallagher Premiership clubs will have flexibil...,business,[],[],0,0,[],0
360615,NaN,2020-12-14,marketscreener,"Don't stockpile food, minister tells British s...",https://www.marketscreener.com/quote/currency/...,"Britain and the EU agreed on Sunday to ""go the...",business,['Tesco PLC'],['Tesco'],1,1,['United Kingdom'],1
348240,Joe Hoppe,2020-11-23,marketwatch,"Wynnstay Properties profit halves, lifts dividend",https://www.marketwatch.com/story/wynnstay-pro...,Wynnstay Properties PLC said Monday that its f...,business,['Wynnstay Properties Plc'],['Wynnstay Properties PLC'],1,1,['United Kingdom'],1
150912,NaN,2020-06-11,finance.yahoo,U.K. Scientists Defy Johnson to Speak Out on V...,https://ca.finance.yahoo.com/news/johnson-unde...,(Bloomberg) -- U.K. Prime Minister Boris Johns...,business,[],[],0,0,[],0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
169245,By Reuters,2020-06-24,nytimes,"With No Tourists to Watch Migration, Kenyan Op...",https://www.nytimes.com/reuters/2020/06/24/bus...,NAIROBI — For Kenyan-based safari operator Saf...,business,[],[],0,0,[],0
306948,NaN,2020-09-12,marketscreener,"Pfizer, BioNTech propose expanding COVID-19 va...",https://www.marketscreener.com/news/latest/Pfi...,Pfizer Inc and BioNTech SE\non Saturday propos...,business,['Pfizer Inc.'],['Pfizer Inc'],1,1,['United States'],1
111840,NaN,2020-05-13,marketscreener,Ameresco : Announces Commercial Operations of ...,https://www.marketscreener.com/AMERESCO-INC-64...,Beale Hill Wind Farm is the first renewable ge...,business,"['Ameresco, Inc.']","['Ameresco, Inc.']",1,1,['United States'],1
72139,NaN,2020-04-16,marketscreener,"L'Oréal: News release: ""First Quarter 2020 Sales""",https://www.marketscreener.com/quote/stock/L-O...,"Clichy, 16 April 2020 at 6:00 p.m. First Quart...",business,"[""L'Oréal S.A.""]","[""L'Oréal""]",1,1,['France'],1


In [11]:
ner_processor = en_core_web_sm.load()

In [1]:
['AFP', 'Associated Press', 'BBC', 'Bloomberg', 'CNN', 'Daily Express', 'Ducker Frontier', 'Financial Times', 'Guardian', 'Marketwatch', 'NBC', 
 'Prudent Speculator', 'Reuters', 'RNZ', 'Sky News', 'SpreadEx', 'The Daily Mail', 'The Sun', 'The Canadian Press', 'LBC', 'New York Times'] # Reuters, New York Times also listed

['Aberdeen Standard Investments', 'Australian Bureau of Statistics', 'Commonwealth Financial Network', 'Federal Reserve', 'National Health Service', 
 'National Rail', 'New York Stock Exchange' 'NHS', 'Oxford University', 'Scientific Advisory Group for Emergencies', 'Securities and Exchange Commission', 
 'The Bank of England', 'The British Retail Consortium', 'The Office for Budget Responsibility', 'The Organisation for Economic Co-operation and Development', 
 'The Reserve Bank of Australia', 'The Vaccine Group', 'Toronto Stock Exchange', 'University of Plymouth', 'Eastspring Investments', 'NATO', 'FTSE', 'SEC', 
 'CDC', 'NYSE', 'C.D.C', 'W.H.O']
 
 # Exclude as often commentators or index related
['Dow', 'Nasdaq', 'Evercore ISI']

['Dow', 'Nasdaq', 'Evercore ISI']

In [331]:
df_lab[df_lab['content'].str.contains("Evercore")]

,author,date,domain,title,url,content,topic_area,org_names_listed,org_names_listed_uo
358782,Ben Levisohn,2020-12-10,marketwatch,Dow Jones Industrial Average Slips as Airbnb S...,https://www.marketwatch.com/articles/dow-jones...,Talk about leaving money on the table. The maj...,business,[],[]
7221,Greg Robb,2020-02-24,marketwatch,Spread of coronavirus lifts odds for Fed rate ...,https://www.marketwatch.com/story/spread-of-co...,"Recent global developments, including the more...",business,['CME Group Inc.'],['CME Group']
199564,Thomas Franck@tomwfranck,2020-07-10,cnbc,Stocks making the biggest moves midday: Gilead...,https://www.cnbc.com/2020/07/10/stocks-making-...,Check out the companies making headlines in mi...,finance,"[Gilead Sciences, Inc., Carnival Corporation &...","[Gilead Sciences, Carnival Corp., American Air..."
276818,NaN,2020-08-17,finance.yahoo,Coronavirus stimulus: South Dakota governor de...,https://finance.yahoo.com/news/coronavirus-sti...,South Dakota is the first state to decline the...,business,[],[]
344877,Jesse Pound@jesserpound,2020-11-17,cnbc,Walgreens drops 9% to lead drug store stocks l...,https://www.cnbc.com/2020/11/17/walgreens-drop...,Pharmacy stocks fell sharply on Tuesday after ...,finance,NaN,NaN


In [77]:
#df_lab.loc[117743,'org_names_listed']

In [80]:
#df_lab.loc[117743,'listing_country']

In [329]:
df_lab.loc[199564,'content']

'Check out the companies making headlines in midday trading: Gilead Sciences — Gilead gained 2.1% after saying its coronavirus treatment candidate drug, remdesivir, was associated with an improvement in clinical recovery and a 62% reduction in the risk of mortality compared with standard of care in a trial. Carnival Corp. — Shares of the cruise line operator jumped 8.2% after the company said it is seeing strong demand for trips in 2021. Carnival also said it can break even on cash flow with capacity running between 30% and 50%. United, American Airlines — Shares of major U.S. airline companies rallied in unison after positive news about a coronavirus treatment raised hopes for a smooth economic reopening. American Airlines and Delta Air Lines rose 4.6% and 4.0%, respectively, while United Airlines jumped 5.5%. Redfin — The real-estate company dropped 8.2% in midday trading after RBC Capital Markets downgraded the stock to sector perform from outperform. The brokerage cited an overly a

In [487]:
df_lab[df_lab['org_names_listed'].isna()]

,author,date,domain,title,url,content,topic_area,org_names_listed,org_names_listed_uo
239166,Al Root,2020-07-28,marketwatch,Tesla Filing Reveals More About Electric Vehic...,https://www.marketwatch.com/articles/teslas-10...,Justin Sullivan/Getty Images The Tesla 10-Q re...,business,NaN,NaN
344877,Jesse Pound@jesserpound,2020-11-17,cnbc,Walgreens drops 9% to lead drug store stocks l...,https://www.cnbc.com/2020/11/17/walgreens-drop...,Pharmacy stocks fell sharply on Tuesday after ...,finance,NaN,NaN


In [13]:
num = 999
article_index = article_indices[num]
article_index = 241927
print(article_index)
print(df_lab.loc[article_index]['org_names_listed'])

article = df_lab.loc[article_index]['content']
spacy.displacy.render(ner_processor(article), style='ent', options = {"ents": ["ORG"]})
print(article_index)

241927
[]


241927


In [ ]:
listed_orgs =    []
listed_orgs_uo = []

In [60]:
article_index = 202383
listed_orgs =    ['Beiersdorf Aktiengesellschaft', 'Dabur India Limited', 'Emami Limited', 'Hindustan Unilever Limited', 'Kao Corporation', 
                 'Shiseido Company, Limited', 'The Procter & Gamble Company']
listed_orgs_uo = ['Beiersdorf AG', 'Dabur India Limited', 'Emami Limited', 'Hindustan Unilever Ltd', 'Kao Corporation', 'Shiseido Company Limited',
                  'The Procter & Gamble Company']

df_lab.at[article_index,'org_names_listed'] = listed_orgs
df_lab.at[article_index,'org_names_listed_uo'] = listed_orgs_uo

In [495]:
df_lab.loc[article_indices[990:1000]]

,author,date,domain,title,url,content,topic_area,org_names_listed,org_names_listed_uo
208896,NaN,2020-07-15,finance.yahoo,VW brand freezes new hiring until end 2020,https://uk.finance.yahoo.com/news/vw-brand-fre...,Volkswagen <VOWG_p.DE> has stopped hiring new ...,business,[Volkswagen AG],[Volkswagen]
178440,NaN,2020-06-30,finance.yahoo,Coronavirus news – live: Police in dark over h...,https://www.finance.yahoo.com/news/coronavirus...,Non-essential shops have shut again in Leicest...,business,[],[]
52903,NaN,2020-04-02,independent,California Public Health Officials Release Gui...,https://www.independent.com/2020/04/02/califor...,Social distancing and hand washing are still b...,general,[],[]
300608,Matthew J. Belvedere@Matt_Belvedere,2020-09-08,cnbc,5 things to know before the stock market opens...,https://www.cnbc.com/2020/09/08/5-things-to-kn...,Tech stocks extended their worst rout since Ma...,finance,"[SoftBank Corp., General Motors Company, Pfize...","[Softbank, General Motors, Pfizer, GlaxoSmithK..."
181700,NaN,2020-07-01,finance.yahoo,Coronavirus: US buys nearly all of Gilead's Co...,https://ca.finance.yahoo.com/news/coronavirus-...,The US is buying nearly all the next three mon...,business,"[Gilead Sciences, Inc.]",[Gilead]
169245,By Reuters,2020-06-24,nytimes,"With No Tourists to Watch Migration, Kenyan Op...",https://www.nytimes.com/reuters/2020/06/24/bus...,NAIROBI — For Kenyan-based safari operator Saf...,business,[],[]
306948,NaN,2020-09-12,marketscreener,"Pfizer, BioNTech propose expanding COVID-19 va...",https://www.marketscreener.com/news/latest/Pfi...,Pfizer Inc and BioNTech SE\non Saturday propos...,business,[Pfizer Inc.],[Pfizer Inc]
111840,NaN,2020-05-13,marketscreener,Ameresco : Announces Commercial Operations of ...,https://www.marketscreener.com/AMERESCO-INC-64...,Beale Hill Wind Farm is the first renewable ge...,business,"[Ameresco, Inc.]","[Ameresco, Inc.]"
72139,NaN,2020-04-16,marketscreener,"L'Oréal: News release: ""First Quarter 2020 Sales""",https://www.marketscreener.com/quote/stock/L-O...,"Clichy, 16 April 2020 at 6:00 p.m. First Quart...",business,[L'Oréal S.A.],[L'Oréal]
269305,Quentin Fottrell,2020-08-13,marketwatch,Fauci on why Asian countries and island nation...,https://www.marketwatch.com/story/dr-fauci-on-...,"Dr. Anthony Fauci, director of the National In...",business,"[Pfizer Inc., GlaxoSmithKline plc, Johnson & J...","[Pfizer, GlaxoSmithKline, Johnson & Johnson, M..."


In [496]:
#df_lab.loc[article_index]

# Articles not done - earnings call transcripts
[191712]

[191712]

In [498]:
file_latest = 'df_lab_2021_07_15_17_10.csv' #999
df_check = pd.read_csv('./data/' + file_latest, index_col=0)
print(sum(df_check['org_names_listed'].isna()))
df_check

0


,author,date,domain,title,url,content,topic_area,org_names_listed,org_names_listed_uo
272801,NaN,2020-08-15,finance.yahoo,Julen Lopetegui: We must be at our best to bea...,https://finance.yahoo.com/news/julen-lopetegui...,Head coach Julen Lopetegui knows Sevilla will ...,business,[],[]
200452,NaN,2020-07-11,finance.yahoo,Gallagher Premiership teams will not be punish...,https://uk.finance.yahoo.com/news/gallagher-pr...,Gallagher Premiership clubs will have flexibil...,business,[],[]
360615,NaN,2020-12-14,marketscreener,"Don't stockpile food, minister tells British s...",https://www.marketscreener.com/quote/currency/...,"Britain and the EU agreed on Sunday to ""go the...",business,['Tesco PLC'],['Tesco']
348240,Joe Hoppe,2020-11-23,marketwatch,"Wynnstay Properties profit halves, lifts dividend",https://www.marketwatch.com/story/wynnstay-pro...,Wynnstay Properties PLC said Monday that its f...,business,['Wynnstay Properties Plc'],['Wynnstay Properties PLC']
150912,NaN,2020-06-11,finance.yahoo,U.K. Scientists Defy Johnson to Speak Out on V...,https://ca.finance.yahoo.com/news/johnson-unde...,(Bloomberg) -- U.K. Prime Minister Boris Johns...,business,[],[]
...,...,...,...,...,...,...,...,...,...
169245,By Reuters,2020-06-24,nytimes,"With No Tourists to Watch Migration, Kenyan Op...",https://www.nytimes.com/reuters/2020/06/24/bus...,NAIROBI — For Kenyan-based safari operator Saf...,business,[],[]
306948,NaN,2020-09-12,marketscreener,"Pfizer, BioNTech propose expanding COVID-19 va...",https://www.marketscreener.com/news/latest/Pfi...,Pfizer Inc and BioNTech SE\non Saturday propos...,business,['Pfizer Inc.'],['Pfizer Inc']
111840,NaN,2020-05-13,marketscreener,Ameresco : Announces Commercial Operations of ...,https://www.marketscreener.com/AMERESCO-INC-64...,Beale Hill Wind Farm is the first renewable ge...,business,"['Ameresco, Inc.']","['Ameresco, Inc.']"
72139,NaN,2020-04-16,marketscreener,"L'Oréal: News release: ""First Quarter 2020 Sales""",https://www.marketscreener.com/quote/stock/L-O...,"Clichy, 16 April 2020 at 6:00 p.m. First Quart...",business,"[""L'Oréal S.A.""]","[""L'Oréal""]"


In [81]:
df_lab_latest = df_lab.copy()
df_lab_latest.to_csv('./data/df_lab_latest.csv')

# Add listing country

In [82]:
df_company_names = pd.read_excel('./data/company_names_listed.xlsx')

In [83]:
df_company_names.tail(10)

,id,name,common_names,ticker_symbol,country,industry,subindustry
51254,636398979,LFG Investment Holdings Limited,NaN,3938,Hong Kong,Financials,Capital Markets
51255,637316385,National World Plc,NaN,NWOR,United Kingdom,Consumer Discreptionary,Media
51256,638075445,"Pacific Conquest Holdings, Inc.",NaN,PCHK,United States,Materials,Metals and Mining
51257,new001,"SelectQuote, Inc",NaN,SLQT,United States,Financials,Insurance
51258,new002,Howmet Aerospace Inc.,NaN,HWM,United States,Industrials,Aerospace and Defense
51259,new003,"Revolution Medicines, Inc.",NaN,RVMD,United States,Healthcare,Biotechnology
51260,new004,PG&E Corporation,Pacific Gas and Electric Co,PCG,United States,Utilities,Multi-Utilities
51261,new005,"Virgin Galactic Holdings, Inc.",NaN,SPCE,United States,Industrials,Aerospace and Defense
51262,new006,"Rocket Companies, Inc.",NaN,RKT,United States,Financials,Thrifts and Mortgage Finance
51263,new007,Carrier Global Corporation,NaN,CARR,United States,Industrials,Building Products


In [84]:
df_lab['listing_country'] = ''
col_index = df_lab.columns.get_loc('listing_country')
for idx, row in enumerate(df_lab['org_names_listed']):
    countries = []
    for name in row:
        try:
            country = df_company_names[df_company_names['name'] == name]['country'].values[0]
            countries.append(country)
        except:
            pass
    df_lab.iat[idx,col_index] = countries

In [85]:
df_lab_latest = df_lab.copy()
df_lab_latest.to_csv('./data/df_lab_latest.csv')

# Check completed file

In [86]:
df_lab = pd.read_csv('./data/df_lab_latest.csv', index_col=0, converters={'org_names_listed': eval, 'org_names_listed_uo': eval, 'listing_country': eval})

In [87]:
# Checl count of labelled columns
df_lab['listed_count'] = df_lab['org_names_listed'].str.len()
df_lab['listed_uo_count'] = df_lab['org_names_listed_uo'].str.len()
df_lab['listing_country_count'] = df_lab['listing_country'].str.len()

In [88]:
df_lab_latest = df_lab.copy()
df_lab_latest.to_csv('./data/df_lab_latest.csv')

In [89]:
df_lab[df_lab['listed_count'] != df_lab['listed_uo_count']]

,author,date,domain,title,url,content,topic_area,org_names_listed,org_names_listed_uo,listed_count,listed_uo_count,listing_country,listing_country_count


In [90]:
df_lab[df_lab['listed_count'] != df_lab['listing_country_count']]

,author,date,domain,title,url,content,topic_area,org_names_listed,org_names_listed_uo,listed_count,listed_uo_count,listing_country,listing_country_count


In [27]:
df_lab.loc[426]

author                                                           Jaimy Lee
date                                                            2020-01-23
domain                                                         marketwatch
title                    Vaccine makers tap into virus-driven rally to ...
url                      https://www.marketwatch.com/story/vaccine-make...
content                  Some small vaccine developers that saw their s...
topic_area                                                        business
org_names_listed         [Novavax, Inc., Moderna, Inc., NanoViricides, ...
org_names_listed_uo       [Novavax Inc., Moderna Inc., NanoViricides Inc.]
listed_count                                                             3
listed_uo_count                                                          3
listing_country                                                         []
listing_country_count                                                    0
Name: 426, dtype: object